# Démo projet Combinatoire

In [1]:
from IPython.display import HTML
import Rules as R
import Grammars as G
import GrammarsC as GC
import CRules as CR


In [2]:

class IncorrectGrammar(Exception):

    def __init__(self, message):
        super(IncorrectGrammar, self).__init__(message)
        
'''Vérifie que tous les non-terminaux apparaissant dans 
les règles sont bien définis dans la grammaire '''
def check_defined_rules (grammar):
    for rule_id in grammar:
        rule = grammar[rule_id]
        if isinstance(rule, R.ConstructorRule):
            fst, snd = rule._parameters
            if not (fst in grammar):
                raise IncorrectGrammar(fst+" rule not defined in grammar")
            if not ( snd in grammar):
               raise IncorrectGrammar(snd+" rule not defined in grammar")

            
# Sets grammar for each rule, computes valuations, checks grammar
def init_grammar (grammar):

    for rule_id in grammar:
        grammar[rule_id]._set_grammar(grammar)

    check_defined_rules(grammar)

    ''' Pour le calcul des valuations on a besoin de comparer 
    les résultats de l'itération courante avec ceux de l'itération précédente,
    on stocke donc les valuations calculées dans les dictionnaires val_before 
    et val_after'''

    val_before, val_after = {}, {}

    for rule in grammar.values():
        if isinstance(rule, R.ConstructorRule):
            val_before[rule] = rule.valuation() 
            val_after [rule] = 0

    # Tant que l'on n'a pas atteint le point fixe, on met à jour les valuations 
    while (val_before != val_after):
        val_before = val_after.copy()
        for rule in val_after:
            rule._update_valuation()
            val_after[rule] = rule.valuation()
   
    # Si on atteint un point fixe pour lequel une des règles à une valuation infine,
    # la grammaire est mal construite
    for rule_id in grammar:
        if grammar[rule_id].valuation() == float('inf'):
            raise IncorrectGrammar("Rule "+rule_id+" is incorrect (inf valuation)")

   
# Récupérer un dictionnaire associant à chaque règle de la 
#  grammaire sa valuation
def get_valuation(gram):
    d = {}
    for rule_name in gram:
        d[rule_name] = gram[rule_name].valuation()
    return d



On récupère un dictionnaire qui contient toutes les grammaires définies,
puis on vérifie que tous les non-terminaux de la grammaire sont bien définis (fonction check_defined_rules), on calcule les valuations et on initialise les grammaires (fonction init_grammar)

In [3]:
grammars = G.grammars

for g in grammars:
    check_defined_rules(grammars[g][0])
    init_grammar(grammars[g][0])


In [4]:
def tests (name, gram, rule_init, card_fun, n, valuation = []):
    print ("\nTests on "+name)
    print("\n")
    #G.print_grammar(gram)
    rule = gram[rule_init]
    try:
        
        # V A L U A T I O N 

        if len(valuation) != 0:
            print ("Valuation:")
            assert (valuation == get_valuation(gram))
            display(HTML("<p style=\"color:green;\">Passed</p>"))

        # C A R D I N A L I T Y

        print ("\nCardinality:")
        for i in range(n):
            count = rule.count(i)
            assert (count == card_fun(i))
            assert (count == len(rule.list(i)))
        display(HTML("<p style=\"color:green;\">Passed</p>"))

        # R A N K

        try:
            print ("\nRank:")
            assert([rule.rank(i) for i in rule.list(n)] == list(range(rule.count(n))))
            display(HTML("<p style=\"color:green;\">Passed</p>"))
        except NotImplementedError:
            print ("Rank not available for this grammar")
            
        # U N R A N K

        print("\nUnrank:")
        assert(rule.list(n) == [rule.unrank(n, i) for i in list(range(rule.count(n)))])
        display(HTML("<p style=\"color:green;\">Passed</p>"))

        # V A L U A T I O N  &  C O U N T

        ''' La valuation correspond au plus petit mot qu'une règle peut produire :
        on vérifie donc pour chaque règle que la valeur de la valuation calculée correspond au plus 
        petit produit par la règle'''
        
        print("\nValuation & Count:")
        valuation = get_valuation(gram)
        for rule_name in gram:
            i = 0
            while gram[rule_name].count(i) == 0:
                i += 1
            assert(i == valuation[rule_name])
        display(HTML("<p style=\"color:green;\">Passed</p>"))
        
    except AssertionError:
        display(HTML("<p style=\"color:red;\">Not passed</p>"))


In [5]:
Test_Size = 10

for g in grammars:
    tests(g, grammars[g][0], grammars[g][1], grammars[g][2], Test_Size)



Tests on TwoGram



Cardinality:
('UnionRule(Vide, CasABu)', 'ProductRule(AtomB, CasAu)')
('EpsilonRule()', 'ProductRule(AtomA, CasBu)')
('UnionRule(Vide, CasABu)', 'ProductRule(AtomB, CasAu)')
('EpsilonRule()', 'ProductRule(AtomA, CasBu)')
('ProductRule(AtomB, CasAu)', 'EpsilonRule()')
('EpsilonRule()', 'ProductRule(AtomA, CasBu)')
('UnionRule(Vide, CasABu)', 'ProductRule(AtomB, CasAu)')
('EpsilonRule()', 'ProductRule(AtomA, CasBu)')
('ProductRule(AtomB, CasAu)', 'EpsilonRule()')
('EpsilonRule()', 'ProductRule(AtomA, CasBu)')
('ProductRule(AtomB, CasAu)', 'EpsilonRule()')
('EpsilonRule()', 'ProductRule(AtomA, CasBu)')
('ProductRule(AtomB, CasAu)', 'EpsilonRule()')
('EpsilonRule()', 'ProductRule(AtomA, CasBu)')
('UnionRule(Vide, CasABu)', 'ProductRule(AtomB, CasAu)')
('EpsilonRule()', 'ProductRule(AtomA, CasBu)')
('ProductRule(AtomB, CasAu)', 'EpsilonRule()')
('EpsilonRule()', 'ProductRule(AtomA, CasBu)')
('ProductRule(AtomB, CasAu)', 'EpsilonRule()')
('EpsilonRule()', 'ProductRule(At

NameError: global name 'display' is not defined

TODO:
 
    POUR LA DEMO :
    
    -> Améliorer affichage résultats tests
    
    -> Ajouter proprement la grammaire even
    
    -> Ajouter exemples list count rank unrank

    -> Ajouter les valuations calculées à la main pour les tests
    
    -> Pour rank montrer un exemple de définition de grammaire avec les fonctions ajoutées.
    
    -> Pour les grammaires condensées, relancer les tests
    
    -> Test grammaires condensées <=> grammaires développées
    
    -> Constructeur Bound -> Redéfinir les test + quelques exemples
    
    -> Constructeur Sequence -> exemple grammaire de Dyck
    
    -> Ajouter grammaire HTML ?
    
    -> Tests ajouter cas <0 ??
    
    POUR LE RAPPORT :
    
    -> Ajouter definition grammaire even
    
    -> Ajouter valuation even
    
    -> Check defined rules
    
    -> Spécification tests
    
    -> Précisions algos rank unrank count list + exemmple pour rank
    
    -> Ajout grammaires condensées
    
    -> Bound définitions
    
    -> Sequence définitions
    

In [6]:
gc = GC.grammars["EvenGram"][0]
print(gc)
gc = CR.dvp_gram(gc)
print(gc)
init_grammar(gc)
d = {}
l = (gc["U"].count(0))
print(l)
for mot in l:
    if mot in d:
        print ("PB", mot)
    else:
        d[mot]= 0

{'baT': <CRules.Prod object at 0x7f741c542a10>, 'AB': <CRules.Prod object at 0x7f741c5428d0>, 'BA': <CRules.Prod object at 0x7f741c542a90>, 'S3': <CRules.Union object at 0x7f741c542c50>, 'S2': <CRules.Union object at 0x7f741c542d10>, 'S': <CRules.Union object at 0x7f741c542d50>, 'U': <CRules.Union object at 0x7f741c542b50>, 'T': <CRules.Union object at 0x7f741c5429d0>, 'Sb': <CRules.Prod object at 0x7f741c542850>, 'Sa': <CRules.Prod object at 0x7f741c542950>, 'aSb': <CRules.Prod object at 0x7f741c542cd0>, 'abT': <CRules.Union object at 0x7f741c542bd0>, 'bSa': <CRules.Prod object at 0x7f741c5427d0>}
{'BA': <Rules.ProductRule object at 0x7f741c594ad0>, 'S3': <Rules.UnionRule object at 0x7f741c594b10>, 'S2': <Rules.UnionRule object at 0x7f741c594b50>, 'abT': <Rules.UnionRule object at 0x7f741c594e50>, '20': <Rules.SingletonRule object at 0x7f741c594d50>, '21': <Rules.SingletonRule object at 0x7f741c594dd0>, '22': <Rules.SingletonRule object at 0x7f741c594e90>, 'baT': <Rules.ProductRule ob

RuntimeError: maximum recursion depth exceeded